## ANLY 5336 Analytics Project: COVID-19 Forecasting
Author: John Courtright

This project works through data from the New York Time's COVID-19 datasets. Link: https://github.com/nytimes/covid-19-data

This notebook contains pulls data related to Travis County, Texas from the datasets. Travis County has complete data ranging from 03-13-2020 through 03-13-2023. The county intiailly reported cases and deaths daily, but shifted to report weekly. We will need to resample the data after compiling to account for this.  

In [10]:
# Load data sets
import pandas as pd

df_2020 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2020.csv')
df_2021 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2021.csv')
df_2022 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2022.csv')
df_2023 = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties-2023.csv')
# df_counties = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/rolling-averages/us-counties.csv')
df_mask = pd.read_csv('https://raw.githubusercontent.com/nytimes/covid-19-data/refs/heads/master/mask-use/mask-use-by-county.csv')

In [11]:
df_2020.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 890361 entries, 0 to 890360
Data columns (total 10 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   date                 890361 non-null  object 
 1   geoid                890361 non-null  object 
 2   county               890361 non-null  object 
 3   state                890361 non-null  object 
 4   cases                890361 non-null  int64  
 5   cases_avg            890361 non-null  float64
 6   cases_avg_per_100k   877322 non-null  float64
 7   deaths               890361 non-null  int64  
 8   deaths_avg           890361 non-null  float64
 9   deaths_avg_per_100k  877322 non-null  float64
dtypes: float64(4), int64(2), object(4)
memory usage: 67.9+ MB


In [12]:
df_mask.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3142 entries, 0 to 3141
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   COUNTYFP    3142 non-null   int64  
 1   NEVER       3142 non-null   float64
 2   RARELY      3142 non-null   float64
 3   SOMETIMES   3142 non-null   float64
 4   FREQUENTLY  3142 non-null   float64
 5   ALWAYS      3142 non-null   float64
dtypes: float64(5), int64(1)
memory usage: 147.4 KB


In the yearly data sets, Travis county is referred to by name along with a GEO-ID code "USA-48453". The county's FIPs code in the mask data set is 48453. We want to pull the rows from the yearly data sets that contains only Travis county so that we'll have one, vertically stacked data frame with Travis' data from 03-13-2020 through 03-13-2023. 

In [13]:
# Create indiviudal data frames for Travis county from each year
df2020_travis = df_2020[df_2020['county'] == 'Travis']
df2021_travis = df_2021[df_2021['county'] == 'Travis']
df2022_travis = df_2022[df_2022['county'] == 'Travis']
df2023_travis = df_2023[df_2023['county'] == 'Travis']

# Stack the yearly Travis county data frames into one
df_travis = pd.concat([df2020_travis, df2021_travis, df2022_travis, df2023_travis], ignore_index=True)
# Reset index
df_travis.reset_index(drop=True, inplace=True)
df_travis.head(50)

,date,geoid,county,state,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,2020-03-13,USA-48453,Travis,Texas,4,0.57,0.04,0,0.00,0.00
1,2020-03-14,USA-48453,Travis,Texas,0,0.57,0.04,0,0.00,0.00
2,2020-03-15,USA-48453,Travis,Texas,2,0.86,0.07,0,0.00,0.00
3,2020-03-16,USA-48453,Travis,Texas,0,0.86,0.07,0,0.00,0.00
4,2020-03-17,USA-48453,Travis,Texas,0,0.86,0.07,0,0.00,0.00
5,2020-03-18,USA-48453,Travis,Texas,0,0.86,0.07,0,0.00,0.00
6,2020-03-19,USA-48453,Travis,Texas,1,1.00,0.08,0,0.00,0.00
7,2020-03-20,USA-48453,Travis,Texas,14,2.43,0.19,0,0.00,0.00
8,2020-03-21,USA-48453,Travis,Texas,41,7.25,0.57,0,0.00,0.00
9,2020-03-22,USA-48453,Travis,Texas,17,10.43,0.82,0,0.00,0.00


In [14]:
# Resample to weekly data
df_travis['date'] = pd.to_datetime(df_travis['date'])
# Drop geoid, county, state columns since we've already filtered to Travis county
df_travis = df_travis.drop(columns=['geoid', 'county', 'state'])

# NOW resample to weekly data
df_travis_weekly = df_travis.set_index('date').resample('W').sum().reset_index()
df_travis_weekly.head(10)

,date,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,2020-03-15,6,2.00,0.15,0,0.00,0.00
1,2020-03-22,73,23.69,1.87,0,0.00,0.00
2,2020-03-29,100,104.74,8.23,1,0.09,0.00
3,2020-04-05,281,174.80,13.72,5,0.81,0.09
4,2020-04-12,314,300.15,23.56,3,1.69,0.14
5,2020-04-19,334,371.21,29.15,16,3.53,0.28
6,2020-04-26,304,316.67,24.86,14,5.60,0.43
7,2020-05-03,344,318.86,25.04,13,10.13,0.78
8,2020-05-10,371,361.71,28.39,8,11.85,0.93
9,2020-05-17,332,338.72,26.60,17,13.99,1.11


In [15]:
# Round cases, deaths, cases_avg, deaths_avg to nearest 2nd decimal place
df_travis_weekly['cases'] = df_travis_weekly['cases'].round(2)
df_travis_weekly['deaths'] = df_travis_weekly['deaths'].round(2)
df_travis_weekly['cases_avg_per_100k'] = df_travis_weekly['cases_avg_per_100k'].round(2)
df_travis_weekly['deaths_avg_per_100k'] = df_travis_weekly['deaths_avg_per_100k'].round(2)
df_travis_weekly.head(10)

,date,cases,cases_avg,cases_avg_per_100k,deaths,deaths_avg,deaths_avg_per_100k
0,2020-03-15,6,2.00,0.15,0,0.00,0.00
1,2020-03-22,73,23.69,1.87,0,0.00,0.00
2,2020-03-29,100,104.74,8.23,1,0.09,0.00
3,2020-04-05,281,174.80,13.72,5,0.81,0.09
4,2020-04-12,314,300.15,23.56,3,1.69,0.14
5,2020-04-19,334,371.21,29.15,16,3.53,0.28
6,2020-04-26,304,316.67,24.86,14,5.60,0.43
7,2020-05-03,344,318.86,25.04,13,10.13,0.78
8,2020-05-10,371,361.71,28.39,8,11.85,0.93
9,2020-05-17,332,338.72,26.60,17,13.99,1.11


In [16]:
# Pull mask data for Travis county
df_mask_travis = df_mask[df_mask['COUNTYFP'] == 48453]
df_mask_travis.reset_index(drop=True, inplace=True)
df_mask_travis.head()

,COUNTYFP,NEVER,RARELY,SOMETIMES,FREQUENTLY,ALWAYS
0,48453,0.015,0.008,0.024,0.158,0.795


In [17]:
# Create new CSVs based on the processed data frames
df_travis_weekly.to_csv('travis_county_covid_weekly.csv', index=False)
df_mask_travis.to_csv('travis_county_mask_usage.csv', index=False)  